In [1]:
from google.colab import files
uploaded = files.upload()

Saving corpus_hum.txt to corpus_hum.txt
Saving data_paraphraser_norm.csv to data_paraphraser_norm.csv
Saving model.bin to model.bin
Saving paraphrases.xml to paraphrases.xml


**Задание 1**

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 8.4MB/s 


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

In [0]:
 from tqdm import tqdm

**Собственная модель**

In [0]:
data = open('corpus_hum.txt', encoding='utf-8').read().splitlines()

In [7]:
data_norm = [normalize(text) for text in tqdm(data)]

100%|██████████| 139601/139601 [12:46<00:00, 182.18it/s]


In [8]:
data_norm = [text for text in tqdm(data_norm) if text]

100%|██████████| 139601/139601 [00:00<00:00, 1412281.37it/s]


In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

**Модель с rusvectores (ruscorpora_upos_cbow_300_20_2019)**

In [10]:
model2 = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Парафразы**

In [0]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [12]:
data['text_1_norm'] = tqdm(data['text_1'].apply(normalize))

100%|██████████| 7227/7227 [00:00<00:00, 1227613.60it/s]


In [13]:
data['text_2_norm'] = tqdm(data['text_2'].apply(normalize))

100%|██████████| 7227/7227 [00:00<00:00, 1568875.06it/s]


In [14]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=1000,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
def get_embedding(text, model, dim):
    text = text.split()
    
    
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) 
        except (KeyError, ValueError):
            #print(word)
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [16]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [0]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [0]:
y = data['label'].values

In [0]:
clf = svm.SVC(kernel='linear', C=1)

In [0]:
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_macro')

In [22]:
scores

array([0.32236593, 0.32545551, 0.34340106, 0.27778094, 0.24544372])

In [23]:
scores.mean()

0.3028894326441132

In [0]:
data2= pd.read_csv('data_paraphraser_norm.csv')

In [25]:
data2.head(3)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...


In [0]:
dim = 300
X2_text_1_w2v = np.zeros((len(data2['text_1_norm']), dim))
X2_text_2_w2v = np.zeros((len(data2['text_2_norm']), dim))

for i, text in enumerate(data2['text_1_norm'].values):
    X2_text_1_w2v[i] = get_embedding(text, model2, dim)
    
for i, text in enumerate(data2['text_2_norm'].values):
    X2_text_2_w2v[i] = get_embedding(text, model2, dim)

In [27]:
X2_text_1_w2v

array([[-0.01181119,  0.00290708,  0.00843334, ..., -0.0012075 ,
         0.00125098,  0.00586502],
       [-0.01236443, -0.00581011,  0.00909714, ..., -0.01083179,
         0.01784532,  0.02289439],
       [-0.00154341, -0.0011991 ,  0.01663633, ..., -0.00323756,
         0.00876778,  0.00953614],
       ...,
       [-0.00611568,  0.02546726, -0.0051658 , ...,  0.0120282 ,
        -0.00098239,  0.04156633],
       [-0.00904899,  0.00464396,  0.00377784, ..., -0.00119408,
        -0.0029938 ,  0.02085814],
       [-0.00073686,  0.00482349,  0.00482718, ..., -0.01427073,
         0.00296348,  0.04529098]])

In [0]:
X2_text_w2v = np.concatenate([X2_text_1_w2v, X2_text_2_w2v], axis=1)

In [0]:
scores = cross_val_score(clf, X2_text_w2v, y, cv=5, scoring='f1_macro')

In [30]:
scores

array([0.33851747, 0.31530458, 0.32665372, 0.30513345, 0.30135462])

In [31]:
scores.mean()

0.3173927681616112

**Вывод:** предобученная модель с rusvectores показывает более хороший результат, но он оценка не слишком далека от нашей модели.

**Задание 2**

**SVD**

In [0]:
svd = TruncatedSVD(200)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

**NMF**

In [0]:
nmf = NMF(50)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

**Fastext**

In [0]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50, min_n=4, max_n=8) 

In [35]:
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


**W2V собственной модели**

In [36]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


**W2V модели с rusvectores**

In [0]:
dim = 300
X2_text_1_w2v = np.zeros((len(data2['text_1_norm']), dim))
X2_text_2_w2v = np.zeros((len(data2['text_2_norm']), dim))

for i, text in enumerate(data2['text_1_norm'].values):
    X2_text_1_w2v[i] = get_embedding(text, model2, dim)
    
for i, text in enumerate(data2['text_2_norm'].values):
    X2_text_2_w2v[i] = get_embedding(text, model2, dim)

**Cosine distance**

In [38]:
data2['text_1_norm_without_pos'] = tqdm(data2['text_1'].apply(normalize))

100%|██████████| 7227/7227 [00:00<00:00, 1228061.22it/s]


In [39]:
data2['text_2_norm_without_pos'] = tqdm(data2['text_2'].apply(normalize))

100%|██████████| 7227/7227 [00:00<00:00, 1286433.60it/s]


In [40]:
data2.head(2)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_without_pos,text_2_norm_without_pos
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...


**cosine_svd**

In [0]:
distances_svd = np.zeros(data2.shape[0])

In [42]:
len(distances_svd)

7227

In [0]:
for i in range(len(data['text_1_norm'])):
    distances_svd[i] = cosine_distances(X_text_1_svd[[i]], X_text_2_svd[[i]])[0][0]


In [44]:
distances_svd

array([0.97181636, 0.98399647, 0.96520059, ..., 1.12259946, 1.06096951,
       1.04245063])

In [0]:
data2['cosine_svd'] = distances_svd

In [46]:
data2.head(3)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_without_pos,text_2_norm_without_pos,cosine_svd
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.971816
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.983996
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.965201


**cosine_ NMF**

In [0]:
distances_nmf = np.zeros(data2.shape[0])

In [52]:
distances_nmf

array([0.87258658, 0.97867933, 0.96845604, ..., 0.99764299, 0.99791956,
       0.99974351])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_nmf[i]  = cosine_distances(X_text_1_nmf[[i]], X_text_2_nmf[[i]])[0][0]


In [0]:
data2['cosine_ NMF'] = distances_nmf

In [55]:
data2.head(3)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_without_pos,text_2_norm_without_pos,cosine_svd,cosine_ NMF
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.971816,0.872587
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.983996,0.978679
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.965201,0.968456


**cosine_Fastext**

In [0]:
distances_ft = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_ft[i]  = cosine_distances(X_text_1_ft[[i]], X_text_2_ft[[i]])[0][0]

In [0]:
data2['cosine_ ft'] = distances_ft

**cosine W2V собственной модели**

In [0]:
distances_w2v1 = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_w2v1[i]  = cosine_distances(X_text_1_w2v[[i]], X_text_2_w2v[[i]])[0][0]

In [0]:
data2['cosine_w2v_1'] = distances_w2v1

**cosine W2V модели с rusvectores**

In [0]:
distances_w2v2 = np.zeros(data2.shape[0])

In [0]:
for i in range(len(data2['text_1_norm'])):
    distances_w2v2[i]  = cosine_distances(X2_text_1_w2v[[i]], X2_text_2_w2v[[i]])[0][0]

In [0]:
data2['cosine_w2v_2'] = distances_w2v2

In [67]:
data2.head(3)

,label,text_1,text_2,text_1_norm,text_2_norm,text_1_norm_without_pos,text_2_norm_without_pos,cosine_svd,cosine_ NMF,cosine_ ft,cosine_w2v_1,cosine_w2v_2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,0.971816,0.872587,0.240253,0.106296,0.228554
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,0.983996,0.978679,0.229163,0.081106,0.220138
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,0.965201,0.968456,0.406469,0.048448,0.151796


### Кроссвалидация на 5 парах

In [0]:
X_cosine = data2.iloc[:,7:12].values

In [0]:
scores_5 = cross_val_score(clf, X_cosine, y, cv=5, scoring='f1_macro')

In [77]:
scores_5

array([0.43155474, 0.45274335, 0.4694408 , 0.29977963, 0.32704332])